<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# NeurIPS - Run ZSMM

Goal : train models on ZSMM

Data : ZSMM

Models : SelfAttnLNP and ConvLNP and ConvCNP 

Loss : NLLLloss

Runs : 1

In [ ]:
import os

os.chdir("..")

In [ ]:
import submitit

from train_imgs import main, parse_arguments
from results import get_exp_args

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train = True
        return submitit.utils.DelayedSubmission(self, args)

    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)


In [10]:
args = get_exp_args("exp_equivariance", is_load=False)
len(args)

3

In [19]:
executor=submitit.SlurmExecutor(folder="logs/%j", max_num_timeout=3)
executor.update_parameters(num_gpus=1, 
                           time=60*24*2,  
                           cpus_per_task=10, 
                           mem='32GB',
                          partition="priority",
                           comment="neurips",
                           constraint="volta32gb"
                          )

In [20]:
jobs = executor.map_array(Run(), args)

In [21]:
############################################################

In [32]:
jobs

[SlurmJob<job_id=26760187_0, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26760187_1, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26760187_2, task_id=0, state="COMPLETED">]

In [33]:
for job in jobs:
    print("--------------------------------")
    print(job.stdout())

--------------------------------
submitit INFO (2020-05-24 11:10:29,543) - Starting with JobEnvironment(job_id=26760187_0, hostname=learnfair0899, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2020-05-24 11:10:29,544) - Loading pickle: /private/home/yannd/projects/NPF/logs/26760187_0/26760187_0_submitted.pkl

--- Training zs-multi-mnist/ConvNPF_CNPF/run_0 ---

  epoch    train_loss    valid_loss    cp       dur
-------  ------------  ------------  ----  --------
      1    -3395.7554    -3777.7707     +  123.3981
      2    -3828.6315    -3808.9409     +  123.4747
      3    -3812.1691    -3855.3044     +  122.8613
      4    -3856.9804    -3946.1013     +  123.0591
      5    -3895.8177    -3913.0227        122.7985
      6    -3876.4443    -3968.0529     +  122.7332
      7    -3935.7825    -3940.6376        122.1634
      8    -3937.4841    -3955.6491        122.9261
      9    -3969.0630    -3849.1058        122.8861
     10    -3969.2065    -4025.2466     +  122.795

In [10]:
for j in job:
    print("--------------------------------")
    print(j.stderr())

--------------------------------

--------------------------------

  0%|          | 1/1032 [00:00<15:59,  1.07it/s, train_loss=742]
--------------------------------

--------------------------------

--------------------------------
/private/home/yannd/projects/NPF/neuralproc/architectures/mlp.py:67: UserWarning: hidden_size=32 smaller than output=128 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))

--------------------------------
/private/home/yannd/projects/NPF/neuralproc/architectures/mlp.py:67: UserWarning: hidden_size=32 smaller than output=128 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))

--------------------------------
/private/home/yannd/projects/NPF/neuralproc/architectures/mlp.py:67: UserWarning: hidden_size=32 smaller than output=128 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, 

In [18]:
for j in jobs:
    j.cancel()